# Clasificación de imágenes
Para iniciar, se llevará a cabo la tarea de clasificación de imágenes. Esta tarea se basa enteramente en el reconocimiento visual, y el objetivo es asignar a una imagen una única etiqueta sobre la clase a la que pertenece la imagen. Para ello, es necesario utilizar imágenes que contengan un único objeto para que así el algoritmo pueda asignar una etiqueta a la imagen. A continuación, encontrará las funciones necesarias para clasificar una imagen utilizando una red neuronal convolucional sencilla tomada de la librería OpenCV.

In [ ]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from operator import itemgetter 

def cargarClasesImageNet():
    with open(os.path.join("Datos","clases_ImageNet.txt"), "r") as f:
        clases = f.readlines()
    clases = [c.strip().split(",") for c in clases]
    return clases

def crearClasificador():
    arquitectura = os.path.join("Datos","DenseNet_121.prototxt.txt")
    pesos_preentrenados = os.path.join("Datos","DenseNet_121.caffemodel")
    modelo = cv.dnn.readNetFromCaffe(arquitectura, pesos_preentrenados) 
    return modelo

def cargarImagenClasificacion(ruta: str):
    imagen = cv.imread(ruta)
    entrada = cv.dnn.blobFromImage(imagen, 0.007843, (224, 224), (127.5, 127.5, 127.5))
    return imagen, entrada

def clasificar(ruta_imagen: str):
    clases = cargarClasesImageNet()
    modelo = crearClasificador()
    imagen, entrada = cargarImagenClasificacion(ruta_imagen)
    
    modelo.setInput(entrada)
    prediccion = modelo.forward().squeeze()
    softmax = np.exp(prediccion)/np.sum(np.exp(prediccion))
    top = np.flipud(np.argsort(softmax, axis=0)[-3:])
    clases_top=itemgetter (*top)(clases)
    confianzas_top = softmax[top]
    return imagen, clases_top, confianzas_top

def visualizar(imagen: np.array, clases: list, confianzas: list):
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].set_title("Imagen de entrada")
    etiqueta=""
    for i in range(len(confianzas)):
        etiqueta_i = "{}) {} ({}). Confianza: {:.3f}".format(i+1,clases[i][0],", ".join(clases[i][1:]), confianzas[i]) if len(clases[i])>1 else '{}) {}. Confianza: {:.2f}'.format(i+1,clases[i][0],confianzas[i])
        etiqueta += etiqueta_i + "\n"
    ax[0].imshow(imagen[:,:,::-1])
    ax[0].axis("off")
    ax[1].set_title("Top 3: clases identificadas")
    ax[1].axis("off")
    ax[1].text(0,0.5, etiqueta, {'color': 'black', 'fontsize': 13})
    plt.show()

A continuación, vamos a visualizar las imágenes que vamos a clasificar:

In [ ]:
import matplotlib.pyplot as plt 
import skimage.io as io
import cv2 as cv
plt.figure() 
plt.subplot(121) 
plt.imshow(cv.resize(io.imread(os.path.join("Datos","gato.jpg")), (800,800)))
plt.axis("off")
plt.subplot(122) 
plt.imshow(cv.resize(io.imread(os.path.join("Datos","perro.jpg")), (800,800)))
plt.axis("off")
plt.show()

Ahora se utilizarán las funciones definidas anteriormente para realizar la clasificación de cada imagen. Para ello, se debe indicar la ruta del archivo de imagen a la función clasificar() y luego se utiliza la función visualizar() para observar el resultado gráficamente. Se observaran las 3 predicciones con mayor confianza realiazada por el método de deep-learning.

In [ ]:
ruta = os.path.join("Datos","gato.jpg")
imagen, clase, confianza = clasificar(ruta)
visualizar(imagen, clase, confianza)

In [ ]:
ruta = os.path.join("Datos","perro.jpg")
imagen, clase, confianza = clasificar(ruta)
visualizar(imagen, clase, confianza)

## Su turno
Lo invitamos a que experimente cargando una imagen de su preferencia y observe los resultados. Para esto, añada la imagen de su preferencia en la carpeta "Datos" y obtenga la visualización tal y como se hizo con las imágenes anteriores.

# Detección en video

Ahora, se realizará la tarea de detección de objetos en videos. Esta tarea se basa tanto en el reconocimiento como en el agrupamiento visual, pues no solo se busca identificar varios objetos en una imagen, sino que se desea conocer su ubicación, la clase a la que pertenecen y las instancias que hay para ese tipo de objeto. En este caso, la detección se llevará a cabo en un video que simula el caso de los vehículos autónomos. Para esto, se realiza una detección en cada fotograma del video. A continuación, encontrará las funciones necesarias para realizar la tarea de detección utilizando una red neuronal convolucional de la librería OpenCV.

In [ ]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from IPython.display import Video

np.random.seed(20)

def cargarClasesCOCO():
    with open(os.path.join("Datos", "clases_COCO.txt"),"r") as f:
        clases = f.readlines()
    clases.insert(0, "__fondo__")
    colores = np.random.uniform(low=0, high=255, size=(len(clases),3))
    return clases, colores

def crearDetector():
    arquitectura = os.path.join("Datos", "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt")
    pesos_preentrenados = os.path.join("Datos", "frozen_inference_graph.pb")
    modelo = cv.dnn_DetectionModel(pesos_preentrenados, arquitectura)
    modelo.setInputSize(320,320)
    modelo.setInputScale(1.0/127.5)
    modelo.setInputMean((127.5,127.5,127.5))
    modelo.setInputSwapRB(True)
    return modelo

def cargarVideoDeteccion(ruta: str):
    video = cv.VideoCapture(ruta)
    if (video.isOpened()==False):
        print("Error al intentar abrir el archivo...")
    return video

def detectar_Y_generar(ruta_video: str):
    clases, colores = cargarClasesCOCO()
    modelo = crearDetector()
    video = cargarVideoDeteccion(ruta_video)
    ancho = video.get(3)
    alto = video.get(4)
    out = cv.VideoWriter(os.path.join("output_video.mp4"),cv.VideoWriter_fourcc(*'mpv4'), 12, (int(ancho),int(alto)))
    
    tiempo_maximo = time.time() + 60 # 60 segundos de cámara
    (exito, imagen) = video.read()
    print("Procesando video...")
    while exito and time.time()<tiempo_maximo:
        clasesEtiquetasID, confianzas, cajas =modelo.detect(imagen, confThreshold=0.5)
        cajas = list(cajas)
        confianzas = list(np.array(confianzas).reshape(1,-1)[0])
        confianzas = list(map(float, confianzas))
        cajaIds = cv.dnn.NMSBoxes(cajas, confianzas, score_threshold=0.5, nms_threshold=0.2)
        
        for i in cajaIds:
            caja = cajas[np.squeeze(i)]
            confianza_clase = confianzas[np.squeeze(i)]
            confianza_etiquetaID = np.squeeze(clasesEtiquetasID[np.squeeze(i)])
            etiqueta_clase = clases[confianza_etiquetaID]
            claseColor = [int(c) for c in colores[confianza_etiquetaID]]

            texto_imprimir="{} ({:.2f}%)".format(etiqueta_clase.strip(),confianza_clase*100)
            x,y,w,h = caja

            cv.rectangle(imagen, (x,y), (x+w,y+h), color= claseColor, thickness=2)
            cv.putText(imagen, texto_imprimir, (x,y-10), cv.FONT_HERSHEY_PLAIN, 1, claseColor, 2)
        out.write(imagen)
        key = cv.waitKey(1) & 0xFF
        if key== ord("q"):
            break
        (exito, imagen) = video.read()
    video.release()
    out.release()
    cv.destroyAllWindows()

A continuación, vemos el video de prueba para realizar la detección:

In [ ]:
from IPython.display import Video
ruta_origen = os.path.join("Datos","video.mp4")
Video(ruta_origen,width=1000, height=600)

Ahora, se utilizarán las funciones de manera similar a como se hizo en la parte de clasificación, pero esta vez para hacer detección en el video de prueba. Esto puede tardar algunos minutos.

In [ ]:
ruta = os.path.join("Datos","video.mp4")
detectar_Y_generar(ruta)
print("Video procesado exitosamente")

Ahora, veremos el resultado del video. Hemos reducido el número de frames por segundo en el video para que pueda ver claramente las etiquetas predichas por el modelo.

In [ ]:
from IPython.display import Video
ruta_generado = os.path.join("Datos", "output_video.mp4")
Video(ruta_generado,width=1000, height=600)

## Su turno (Opcional)
Si desea probar la detección de video sobre un video propio, utilice las funciones definidas en las celdas anteriores desde su compatdor. Para detectar sobre su video, pase la ruta de su video a la función detectar_Y_generar.